In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [2]:
random.seed(1344)
g = zx.generate.cliffordT(qubits=12, depth=1000, p_t=0.3)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 12 qubits with 1000 gates.
        271 is the T-count
        729 Cliffords among which
        217 2-qubit gates (217 CNOT, 0 other) and
        0 Hadamard gates.


In [3]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
print(zx.Circuit.from_graph(g).split_phase_gates().stats())

'Circuit  on 12 qubits with 667 gates.\n        131 is the T-count\n        536 Cliffords among which\n        215 2-qubit gates (209 CNOT, 6 other) and\n        139 Hadamard gates.'

In [4]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
print(zx.extract_circuit(g_full.copy()).stats())

'Circuit  on 12 qubits with 941 gates.\n        131 is the T-count\n        810 Cliffords among which\n        630 2-qubit gates (133 CNOT, 497 other) and\n        171 Hadamard gates.'

In [5]:
for la in range(0,3):
    print("Lookahead:", la)
    start_time = time.time()
    g_greedy_la = g_tele.copy()
    zx.simplify.greedy_simp(g_greedy_la, lookahead=la)
    print(zx.extract_circuit(g_greedy_la.copy()).stats())
    print("Time:", time.time()-start_time)
    print("---------------------------------")

Lookahead: 0
Circuit  on 12 qubits with 811 gates.
        131 is the T-count
        680 Cliffords among which
        225 2-qubit gates (3 CNOT, 222 other) and
        333 Hadamard gates.
Time: 0.5796630382537842
---------------------------------
Lookahead: 1
Circuit  on 12 qubits with 827 gates.
        131 is the T-count
        696 Cliffords among which
        232 2-qubit gates (3 CNOT, 229 other) and
        335 Hadamard gates.
Time: 41.815192222595215
---------------------------------
Lookahead: 2


In [7]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
print(zx.extract_circuit(g_nu.copy()).stats())

'Circuit  on 8 qubits with 369 gates.\n        69 is the T-count\n        300 Cliffords among which\n        105 2-qubit gates (2 CNOT, 103 other) and\n        155 Hadamard gates.'

In [8]:
g_sim_n = g_tele.copy()
zx.simplify.sim_anneal_simp_neighbors(g_sim_n)
print(zx.extract_circuit(g_sim_n.copy()).stats())

final num edges:  268


'Circuit  on 8 qubits with 377 gates.\n        69 is the T-count\n        308 Cliffords among which\n        105 2-qubit gates (2 CNOT, 103 other) and\n        163 Hadamard gates.'

In [9]:
g_sim = g_tele.copy()
zx.simplify.sim_anneal_simp(g_sim)
print(zx.extract_circuit(g_sim.copy()).stats())

'Circuit  on 8 qubits with 384 gates.\n        69 is the T-count\n        315 Cliffords among which\n        110 2-qubit gates (0 CNOT, 110 other) and\n        163 Hadamard gates.'